In [ ]:
from openai import OpenAI
import pandas as pd
import keyring

In [ ]:
MODEL = 'gpt-4o-mini'

In [ ]:
TWEETS_FILEPATH = "D:/GA Tech/DS@GT/CheckThat! Oraganizer Repo/clef2025-checkthat-lab/task4/subtask_4b/subtask4b_query_tweets_dev.tsv"
ABSTRACTS_FILEPATH = "D:/GA Tech/DS@GT/CheckThat! Oraganizer Repo/clef2025-checkthat-lab/task4/subtask_4b/subtask4b_collection_data.pkl"

In [ ]:
token = keyring.get_password("openai", "jschofield8")
client = OpenAI(api_key=token)

In [ ]:
def get_model_response(client, model, dev_content, user_content):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "developer",
                "content": dev_content
             },
            {
                "role": "user",
                "content": user_content
            },
        ],
    )

    return response.choices[0].message.content

## Part 1: Convert Tweets from Informal to Formal Language

In [6]:
def generate_tweet_file(df, model, model_instructions, col_name, remove_columns=False):
    df[col_name] = df["tweet_text"].apply(lambda x: get_model_response(client, model, model_instructions, x))
    
    if remove_columns:
        df = df[['post_id', col_name, 'cord_uid']]
        df.rename(columns={col_name: 'tweet_text'}, inplace=True)
    
    df.to_csv("subtask4b_query_tweets_dev_{}.tsv".format(col_name), index=False, sep='\t')

In [7]:
instructions = "Rewrite the following COVID-related tweet in formal/scientific language using 25 words or less. Do not include hashtags or special characters in your response."

In [8]:
df_tweets = pd.read_csv(TWEETS_FILEPATH, sep='\t')

In [9]:
generate_tweet_file(
                    df=df_tweets, 
                    model=MODEL, 
                    model_instructions=instructions,
                    col_name="formal"
                   )

## Part 2: Summarize Abstracts in 100 Words or Less. Use Formal Language

In [ ]:
def generate_abstract_file(df, model, col_name, model_instructions):
    df[col_name] = df["abstract"].apply(lambda x: get_model_response(client, model, model_instructions, x))

    df["combined"] = df["title"] + " " + df[col_name] + " " + df["authors"].str.split(";").str[:2].str.join(" ") + " " + df["source_x"] + " " + df["journal"]
    
    df.to_pickle("subtask4b_collection_data_{}.pkl".format(col_name), index=False)

In [ ]:
instructions = "Summarize the user input using formal English in 100 words or less."

In [ ]:
df_abstracts = pd.read_pickle(ABSTRACTS_FILEPATH)

In [ ]:
generate_abstract_file(
                       df=df_abstracts, 
                       model=MODEL, 
                       col_name="formal",
                       model_instructions=instructions
                      )